In [66]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import bz2
import _pickle as cPickle
import numpy as np

path = 'folder_with_raw_data'
# 0 - whole dataset, 1 - dataset for subsets, 2 - dataset for subsets with others
create_subset  = 0

df = pd.read_csv(path + 'UNSW_NB15_training-set.csv')


print("Total rows:", df.shape[0])

# remove id and label from data frame
# 44 features
df = df.drop(['id','label',],axis= 1)

# one hot encoding for protocol -> +131 columns
# print(len(df['proto'].unique()))
# print((df['proto'].unique()))
one_hot_proto = pd.get_dummies(df['proto'])
df = df.drop('proto',axis = 1)
df = df.join(one_hot_proto)

if create_subset ==0:
# one hot encoding for service -> +13 columns
# print(len(df['service'].unique()))
    one_hot_service = pd.get_dummies(df['service'])
    df = df.drop('service',axis = 1)
    df = df.join(one_hot_service)

# one hot encoding for state -> +7 columns
# print(len(df['state'].unique()))
one_hot_state = pd.get_dummies(df['state'])
df = df.drop('state',axis = 1)
df = df.join(one_hot_state)

# classes to remove
# Analysis, Backdoors, Shell-code, Worms
# print(df['attack_cat'].unique())

options = ['Analysis', 'Backdoor', 'Shellcode','Worms']
df = df[~df['attack_cat'].isin(options)]

print("No. of rows without some classes:", df.shape[0])
print("Total number of features + target: ", df.shape[1])

# encoding for output - attack_cat
le = LabelEncoder()
le.fit(df['attack_cat'].unique())
df['attack_cat'] = le.transform(df['attack_cat'])

if create_subset  == 1 or create_subset  == 2:
    array = df['service'].unique()
    n = len(array)

    print("Number of instances/attacks for used services: " )

    # remove services with less then 50 instances of both classes

    used_services = 0
    remove_services = [];
    for i in range(n):
        subset = df[df['service'] == array[i]]
        rows = subset.shape[0]
        attack_rows = subset[subset['attack_cat'] != le.transform(['Normal'])[0]].shape[0]
        normal_rows = subset[subset['attack_cat'] == le.transform(['Normal'])[0]].shape[0]
        if normal_rows>50 and attack_rows>50:
            print(array[i], " -> ", subset.shape[0] , ", attacks: ", attack_rows)
            used_services = used_services+1
        else:
            remove_services.append(array[i])

    print("Number of used service: ", used_services )
    print("Number of instances: ", df.shape[0] )

if create_subset  == 0:
    data = df
    # target
    target = np.array(data['attack_cat'])

    # data
    data = data.drop(['attack_cat'],axis= 1)
    print("Final features: ", data.shape[1])

    scaler= MinMaxScaler()
    data = scaler.fit_transform(data)
    f = bz2.BZ2File("unsw_nb15.pbz2", "wb")
    cPickle.dump((data,target),f)
    f.close()

if create_subset  == 1:
    data = df
    data = data[~data['service'].isin(remove_services)]

    # target
    target = np.array(data['attack_cat'])

    # subsets
    subsets = np.array(data['service'])

    # data
    dataS = data.drop(['service'],axis= 1)
    dataS = data.drop(['attack_cat'],axis= 1)

    scaler= MinMaxScaler()
    data = scaler.fit_transform(data)
    print("Final features: ", dataS.shape[1])

    # f = open("unsw_nb15_subsets", "wb")
    f = bz2.BZ2File("unsw_nb15_subsets.pbz2", "wb")
    cPickle.dump((dataS,target,subsets),f)
    f.close()

if create_subset  == 2:
    data = df

    data['service'].replace(dict.fromkeys(remove_services, 'others'), inplace=True )

    # target
    target = np.array(data['attack_cat'])

    # subsets
    subsets = np.array(data['service'])

    # data
    data = data.drop(['attack_cat'],axis= 1)
    data = data.drop(['service'],axis= 1)

    scaler= MinMaxScaler()
    data = scaler.fit_transform(data)
    print("Final features: ", data.shape[1])

    # f = open("unsw_nb15_subsets", "wb")
    f = bz2.BZ2File("unsw_nb15_subsets_new.pbz2", "wb")
    cPickle.dump((data,target,subsets),f)
    f.close()

Total rows: 82332
No. of rows without some classes: 80650
Total number of features + target:  179
Number of instances/attacks for used services: 
-  ->  45516 , attacks:  18141
http  ->  8244 , attacks:  4231
ftp  ->  1550 , attacks:  792
ftp-data  ->  1396 , attacks:  447
smtp  ->  1851 , attacks:  1216
dns  ->  21367 , attacks:  18299
Number of used service:  6
Number of instances:  80650
Final features:  177
